# Loading data and requirements

In [69]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... error
  ERROR: Command errored out with exit status 1:
   command: /Users/wout/Documents/UvA/Thesis/.venv/bin/python -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/hk/wc9r7c9x18z2qz2dt11hknmw0000gn/T/pip-install-xt5_i6l7/sklearn_86ead01a5641489b8325dc7c3d726d1a/setup.py'"'"'; __file__='"'"'/private/var/folders/hk/wc9r7c9x18z2qz2dt11hknmw0000gn/T/pip-install-xt5_i6l7/sklearn_86ead01a5641489b8325dc7c3d726d1a/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/hk/wc9r7c9x18z2qz2dt11hknmw0000gn/T/pip-pip-egg-info-mezzqvhu
       cwd: /private/var/folders/hk/wc9r7c9x18z2qz2dt11hknmw0000gn/T/pip-install-xt5_i6l7/sklearn_86ead01a5641489b8325dc7c3d726d1a/
  Compl

In [70]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [84]:
df = pd.read_csv('../paysim.csv')

# Data preparation

In [85]:
df = df.rename(columns={'oldbalanceOrg': 'oldbalanceOrig'})

In [86]:
# Set new balance and original balance based on transaction amount based on EDA
# Percentage of observations with balance errors in the account giving money:  85.0
# Percentage of observations with balance errors in the account receiving money:  100.0

df['newbalanceDest'] = df['oldbalanceDest'] + df['amount']
df['oldbalanceOrig'] = df['newbalanceOrig'] + df['amount']

In [87]:
# # Only 6 true
# df['externalDest'] = ((df['oldbalanceDest'] == 0) & (df['newbalanceDest'] == 0)).astype(int)
# # Only 16 true
# df['externalOrig'] = ((df['oldbalanceOrig'] == 0) & (df['newbalanceOrig'] == 0)).astype(int)

# Feature engineering

In [88]:
# Extracting hour of the day from the 'step' column
df['hour'] = df['step']% 24

# Extracting day of the week as integers, add 3 to convert it to correct days of the week (1 = monday, 7 = sunday)
df['weekday'] = (df['step'] // 24) % 7 + 1

# Create is_workday feature based on the 2 least transaction dates being the weekend
df['is_workday'] = df['weekday'].apply(lambda x: 0 if x == 4 or x == 5 else 1)

# Extracting day of the week as integers
df['monthday'] = (df['step'] % 30) + 1

In [89]:
# # calculate the rolling average of last 3 and 7 transactions for each recipient
# df['meanDest3'] = df.groupby('nameDest')['amount'].rolling(window=3, min_periods=1).mean().reset_index(0, drop=True)
# df['meanDest7'] = df.groupby('nameDest')['amount'].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)

# # calculate the rolling maximum of last 3 and 7 transactions for each recipient
# df['maxDest3'] = df.groupby('nameDest')['amount'].rolling(window=3, min_periods=1).max().reset_index(0, drop=True)
# df['maxDest7'] = df.groupby('nameDest')['amount'].rolling(window=7, min_periods=1).max().reset_index(0, drop=True)

In [90]:
# Create a new type column indicatin if transaction was from Customer (C) to Merchant (M) or any other combination

conditions = [
    (df['nameOrig'].str.contains('C')) & (df['nameDest'].str.contains('C')),
    (df['nameOrig'].str.contains('C')) & (df['nameDest'].str.contains('M')),
    (df['nameOrig'].str.contains('M')) & (df['nameDest'].str.contains('C')),
    (df['nameOrig'].str.contains('M')) & (df['nameDest'].str.contains('M'))
]

choices = ['CC', 'CM', 'MC', 'MM']

df['type2'] = np.select(conditions, choices, default=None)

In [91]:
# # One hot encode type columns
df = pd.get_dummies(df, columns=['type', 'type2'])

# Transformation

In [92]:
# Log scale amount
df['log_amount'] = np.log(df['amount'])

/Users/wout/Documents/UvA/Thesis/.venv/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# ML preparation

In [94]:
# Remove unused columns
df.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1, inplace=True)

In [95]:
df

,step,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,hour,weekday,is_workday,monthday,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,type2_CC,type2_CM,log_amount
0,1,9839.64,170136.00,160296.36,0.00,9839.64,0,1,1,1,2,0,0,0,1,0,0,1,9.194174
1,1,1864.28,21249.00,19384.72,0.00,1864.28,0,1,1,1,2,0,0,0,1,0,0,1,7.530630
2,1,181.00,181.00,0.00,0.00,181.00,1,1,1,1,2,0,0,0,0,1,1,0,5.198497
3,1,181.00,181.00,0.00,21182.00,21363.00,1,1,1,1,2,0,1,0,0,0,1,0,5.198497
4,1,11668.14,41554.00,29885.86,0.00,11668.14,0,1,1,1,2,0,0,0,1,0,0,1,9.364617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,339682.13,339682.13,0.00,0.00,339682.13,1,23,3,1,24,0,1,0,0,0,1,0,12.735766
6362616,743,6311409.28,6311409.28,0.00,0.00,6311409.28,1,23,3,1,24,0,0,0,0,1,1,0,15.657870
6362617,743,6311409.28,6311409.28,0.00,68488.84,6379898.12,1,23,3,1,24,0,1,0,0,0,1,0,15.657870
6362618,743,850002.52,850002.52,0.00,0.00,850002.52,1,23,3,1,24,0,0,0,0,1,1,0,13.652995


In [97]:
# Create train/val/test set following 0.7/0.15/0.15 split
train_df, test_df = train_test_split(df, test_size=0.3, random_state=0, shuffle=True)
test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=0, shuffle=True)

In [98]:
# reset index
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)